# Mergeall


In [2]:
import sys
sys.path.append('src/')
from dollarparser import parse_dollar_amount
import os
import json
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import glob
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import datetime

import warnings
warnings.filterwarnings("ignore")

sns.set_theme(context='notebook', style='whitegrid')
pd.set_option("display.max_rows", 100)

In [3]:
start_run = datetime.datetime.now()

In [4]:
prompt_name = "major_name_graduation"

In [5]:
for file in glob.glob(f"input_data/batch_results/{prompt_name}.jsonl"):
    print(file)

In [6]:
# Get all .jsonl files in the directory
files = glob.glob(f"input_data/batch_results/{prompt_name}*.jsonl")

for file in files:
    if os.path.isfile(file):  # Check if it's a regular file
        zip_filename = f"{file}.zip"
        # Overwrite the ZIP file if it already exists
        with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
            zipf.write(file, os.path.basename(file))  # Add the file to the zip
        print(f"Zipped: {file} -> {zip_filename}")

Zipped: input_data/batch_results/major_name_graduation_gpt-4o-mini-2024-07-18_output.jsonl -> input_data/batch_results/major_name_graduation_gpt-4o-mini-2024-07-18_output.jsonl.zip
Zipped: input_data/batch_results/major_name_graduation3_gpt-4o-mini-2024-07-18_output.jsonl -> input_data/batch_results/major_name_graduation3_gpt-4o-mini-2024-07-18_output.jsonl.zip
Zipped: input_data/batch_results/major_name_graduation4_gpt-4o-mini-2024-07-18_output.jsonl -> input_data/batch_results/major_name_graduation4_gpt-4o-mini-2024-07-18_output.jsonl.zip
Zipped: input_data/batch_results/major_name_graduation2_gpt-4o-mini-2024-07-18_output.jsonl -> input_data/batch_results/major_name_graduation2_gpt-4o-mini-2024-07-18_output.jsonl.zip


# Data Cleaning

In [7]:
fns = glob.glob(f"input_data/batch_results/{prompt_name}*.jsonl.zip")
fns.sort()
print(fns)

['input_data/batch_results/major_name_graduation2_gpt-4o-mini-2024-07-18_output.jsonl.zip', 'input_data/batch_results/major_name_graduation3_gpt-4o-mini-2024-07-18_output.jsonl.zip', 'input_data/batch_results/major_name_graduation4_gpt-4o-mini-2024-07-18_output.jsonl.zip', 'input_data/batch_results/major_name_graduation_gpt-4o-mini-2024-07-18_output.jsonl.zip']


In [66]:
dfs = []
input_type = 0
for file in tqdm(fns):
    print(file)
    df = pd.read_json(file, lines=True, encoding_errors='replace')
    
    # Check if the file is in 'claude' format based on the presence of 'result' in columns
    if 'claude' in file or any(df.columns.str.contains('result')):
        # Use json_normalize for 'claude' format
        df = pd.json_normalize(
            df.to_dict(orient='records'),
            record_path=['result', 'message', 'content'],
            meta=['custom_id', ['result', 'message', 'model']],
            record_prefix='content.'
        )
        df = df[['custom_id', 'result.message.model', 'content.text']]
        df.columns = ['custom_id', 'model', 'content']
    else:
        # Standard format parsing
        df = pd.json_normalize(
            df.to_dict(orient='records'),
            record_path=['response', 'body', 'choices'],
            meta=[['custom_id'], ['response', 'body', 'model']],
            record_prefix='choices.'
        )
        df = df[['custom_id', 'response.body.model', 'choices.message.content']]
        df.columns = ['custom_id', 'model', 'content']
    
    # Load the seed file and merge with parsed data
    df_seed = pd.read_csv(f"input_data/{prompt_name}_seed.csv")
    df_seed['custom_id'] = 'task-' + df_seed['run_id'].astype(str)
    df_merged = pd.merge(df, df_seed, on='custom_id', how='inner')
    
    # Process content for dollar amounts - moved to step3
    df_merged['input_type'] = input_type 
    input_type += 1 
    
    dfs.append(df_merged)

df_all = pd.concat(dfs)


  0%|                                                     | 0/4 [00:00<?, ?it/s]

input_data/batch_results/major_name_graduation2_gpt-4o-mini-2024-07-18_output.jsonl.zip


 25%|███████████▎                                 | 1/4 [00:02<00:06,  2.09s/it]

input_data/batch_results/major_name_graduation3_gpt-4o-mini-2024-07-18_output.jsonl.zip


 50%|██████████████████████▌                      | 2/4 [00:04<00:04,  2.00s/it]

input_data/batch_results/major_name_graduation4_gpt-4o-mini-2024-07-18_output.jsonl.zip


 75%|█████████████████████████████████▊           | 3/4 [00:05<00:01,  1.94s/it]

input_data/batch_results/major_name_graduation_gpt-4o-mini-2024-07-18_output.jsonl.zip


100%|█████████████████████████████████████████████| 4/4 [00:07<00:00,  1.96s/it]


In [67]:
df_all.sort_values(by=['model','run_id'], inplace=True)
df_all['query_response_raw'] = df_all['content']
df_all.drop(columns=['content', 'query_response'], inplace=True)
a = df_all.head()
a.to_csv("df_all_file.csv")

In [48]:
import re

# Update the parsing function to handle percentages with "XX"
def parse_major_and_percentage_with_fix(entry):
    # Clean and standardize the input string
    entry = re.sub(r'[\-\–\/,]', ',', entry)  # Replace separators with a comma
    entry = re.sub(r'\s+', ' ', entry)        # Normalize spaces
    
    # Replace "XX" in percentages with "00"
    entry = re.sub(r'(\d+)\.XX', r'\1.00', entry, flags=re.IGNORECASE)
    
    # Use a regex to extract the major and percentage
    match = re.match(r'^(.*?)[,]?\s*([0-9]+\.[0-9]+)\s*%', entry)
    if match:
        major, percentage = match.groups()
        return major.strip(), f"{percentage} %"
    return entry, None  # Return original if parsing fails

# Apply the updated parsing function to the column
df_all[['Major_response', 'Percentage']] = df_all['query_response_raw'].apply(
    lambda x: pd.Series(parse_major_and_percentage_with_fix(x))
)


In [49]:
df_all['Percentage'] = df_all['Percentage'].str.replace('%', '').str.strip().astype(float)


In [50]:
df_all['gender'] = df_all['gender'].str.replace('None-Control', 'Gender-Neutral')
df_all['gender'].value_counts()

gender
Woman             65280
Man               65279
Gender-Neutral    65279
Name: count, dtype: int64

In [51]:
len(df_all['custom_id'].unique())

48960

In [52]:
len(df_all)

195838

In [53]:
assert len(df_all)/len(df_all['custom_id'].unique()) == len(df_all['model'].unique())

AssertionError: 

In [54]:
df_all.groupby(['input_type','model','gender'])['model'].count()

input_type  model                   gender        
0           gpt-4o-mini-2024-07-18  Gender-Neutral    16320
                                    Man               16320
                                    Woman             16320
1           gpt-4o-mini-2024-07-18  Gender-Neutral    16320
                                    Man               16320
                                    Woman             16320
2           gpt-4o-mini-2024-07-18  Gender-Neutral    16320
                                    Man               16320
                                    Woman             16320
3           gpt-4o-mini-2024-07-18  Gender-Neutral    16319
                                    Man               16319
                                    Woman             16320
Name: model, dtype: int64

In [55]:
df_all.to_csv(f"processed_data/{prompt_name}_allmodels.csv.zip", index=False)

In [56]:
print("Elapsed time:", datetime.datetime.now() - start_run)

Elapsed time: 1:10:22.799023
